In [ ]:
import numpy as np
import elboflow as ef
import tensorflow as tf
import scipy.stats
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Generate some data for linear regression
np.random.seed(1)
num_samples = 100
num_dims = 2

feature_means = np.random.normal(0, 1, num_dims)
x = ef.add_bias(np.random.normal(0, 1, (num_samples, num_dims)) + feature_means)
theta = np.random.normal(0, 1, num_dims + 1)
predictor = np.dot(x, theta)
tau = np.random.gamma(1)
y = predictor + np.random.normal(0, 1 / np.sqrt(tau), num_samples)

In [ ]:
class LinearModel(ef.Model):
    def __init__(self, x, y, create_optimizer=None, create_session=None):
        super(LinearModel, self).__init__(None, (x, y), create_optimizer, create_session)
    
    def evaluate_log_joint(self, x, y):
        # Define the factors
        q_theta = ef.MultiNormalDistribution(
            ef.get_variable('theta_mean', num_dims + 1),
            ef.get_positive_definite_variable('theta_log_precision', (num_dims + 1, num_dims + 1))
        )
        q_tau = ef.GammaDistribution(
            ef.get_positive_variable('tau_shape', []),
            ef.get_positive_variable('tau_scale', []),
        )
        
        # Compute the likelihood and joint distribution
        log_likelihood = tf.reduce_sum(ef.NormalDistribution.linear_log_likelihood(x, y, q_theta, q_tau))
        log_joint = log_likelihood + ef.GammaDistribution.log_likelihood(q_tau, 1e-3, 1e-3) + \
            ef.MultiNormalDistribution.log_likelihood(q_theta, np.zeros(num_dims + 1), 1e-3 * np.eye(num_dims + 1))
            
        return log_joint, {
            'theta': q_theta,
            'tau': q_tau
        }
    
model = LinearModel(x, y, lambda: tf.train.AdamOptimizer(0.1))
model.run(model.elbo)

In [ ]:
trace = model.optimize(2000, [model.loss], tqdm=tqdm_notebook)
plt.plot(trace[model.loss])
plt.yscale('log')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

model.plot_proba('theta', reference=theta, ax=ax1)
model.plot_proba('tau', reference=tau, ax=ax2)

In [ ]:
model.plot_comparison('theta', theta)

In [ ]:
model.plot_cov('theta')